In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
#Let's try working with OSHA Covid fatality data to get the cities with the most Covid-19 fatalities with a violation.
requests.get("https://www.osha.gov/fatalities/covid-19")

<Response [403]>

In [4]:
#Blocked. I can't find an undocumented API, either.
#Let's use the same data set from my previous project — OSHA data of penalties with fines of $40,000 or more:

#Got this from the cURL converter.
import requests

cookies = {
    '_ga': 'GA1.2.1221182387.1689314175',
    '_gat_GSA_ENOR0': '1',
    '_gat_GSA_ENOR1': '1',
    '_gid': 'GA1.2.1390347906.1689979681',
}

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    # 'Cookie': '_ga=GA1.2.1221182387.1689314175; _gat_GSA_ENOR0=1; _gat_GSA_ENOR1=1; _gid=GA1.2.1390347906.1689979681',
    'Accept-Language': 'en-US,en;q=0.9',
    'If-None-Match': '"64bb0974-247ec5"',
    'Host': 'www.osha.gov',
    'If-Modified-Since': 'Fri, 21 Jul 2023 22:40:52 GMT',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'Referer': 'https://www.osha.gov/enforcement/toppenalties/bystate',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'X-Requested-With': 'XMLHttpRequest',
}

response = requests.get(
    'https://www.osha.gov/sites/default/files/json/enforcement-cases/data-set.json',
    cookies=cookies,
    headers=headers,
)

In [5]:
import json
data = response.json()

In [6]:
import pandas as pd
df = pd.DataFrame(data)

In [7]:
df.head()

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current


In [8]:
#Let's merge the top 5 employers' names that are identical:
employers_list_fixed = []

for cell in df['employer']:
    if "Dollar Tree" in cell.strip().title() or "Family Dollar" in cell.strip().title():
        cell_dt = cell.replace(cell, "Dollar Tree/Family Dollar (similar names merged)")
        employers_list_fixed.append(cell_dt)
    elif "Dollar General" in cell.strip().title() or "Dolgencorp" in cell.strip().title():
        cell_dg = cell.replace(cell, "Dollar General (similar names merged)")
        employers_list_fixed.append(cell_dg)
    elif "Postal Service" in cell.strip().title():
        cell_ps = cell.replace(cell, "U.S. Postal Service (similar names merged)")
        employers_list_fixed.append(cell_ps)
    elif "Target" in cell.strip().title():
        cell_tgt = cell.replace(cell, "Target (similar names merged)")
        employers_list_fixed.append(cell_tgt)
    elif "Kaiser" in cell.strip().title():
        cell_ksr = cell.replace(cell, "Kaiser (similar names merged)")
        employers_list_fixed.append(cell_ksr)
    else:
        employers_list_fixed.append(cell)

In [9]:
df['employers_fixed'] = employers_list_fixed

In [10]:
df[df['employers_fixed'] == "Target (similar names merged)"]

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,employers_fixed
71,10636,11/28/2022,WASILLA,AK,"Target Stores, Inc.",1601188.015,"$62,151.00",Current,Target (similar names merged)
327,9130,11/23/2021,TUSCALOOSA,AL,Target Corporation,1533241.015,"$40,959.00",Current,Target (similar names merged)
379,11431,05/01/2023,JONESBORO,AR,Target Corporation,1633197.015,"$147,334.00",Current,Target (similar names merged)
1559,7889,11/18/2020,FONTANA,CA,Target Corporation,1479936.015,"$50,000.00",Current,Target (similar names merged)
2308,3419,09/23/2019,WATERFORD,CT,Target Corporation,1413674.015,"$83,763.00",Current,Target (similar names merged)
2354,5050,06/14/2019,STAMFORD,CT,Target Corporation,1371131.015,"$168,537.00",Current,Target (similar names merged)
5080,3660,12/31/2019,FRAMINGHAM,MA,Target Corporation,1414036.015,"$113,652.00",Current,Target (similar names merged)
5081,3661,12/31/2019,DANVERS,MA,Target Corporation,1422713.015,"$121,229.00",Current,Target (similar names merged)
5206,5843,05/21/2019,EVERETT,MA,Target Corporation,1381150.015,"$117,452.00",Current,Target (similar names merged)
5359,3694,01/16/2020,BANGOR,ME,Target Corporation,1416498.015,"$95,420.00",Current,Target (similar names merged)


In [11]:
df.loc[10503, 'employers_fixed'] = 'Kaiser Aluminum & Fabricated Products, LLC'

In [12]:
df['Year'] = df['issuance_date'].str.replace(r"^\d\d/\d\d/(\d{4})", r"\1", regex=True).astype(int)

In [13]:
#OK, now I have a somewhat clean data set.
#For this project, I want to make a scrollytelling map.
#Let's play around with the location data a bit.

In [14]:
df.head()

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,employers_fixed,Year
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current,"Peter Pan Seafood Company, LLC",2023
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current,"Bering Straits Technical Services, LLC",2023
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current,Trident Seafoods Corporation,2023
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current,Prince William Sound Aquaculture Corporation,2023
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current,Matanuska-Susitna Borough School District,2023


In [15]:
#df["Year"][df["employers_fixed"] == "Dollar Tree/Family Dollar (similar names merged)"].value_counts()

In [16]:
#Here's the code to use multiple filters for any column to figure out what occurred the most.
#e.g., Let's say we want to find out in which state Dollar General had the most violations for each year:
print(df["state"][(df["Year"] == 2015) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2016) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2027) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2018) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2019) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2020) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2021) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2022) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())
print(df["state"][(df["Year"] == 2023) &
   (df["employers_fixed"] == "Dollar General (similar names merged)")].value_counts())

state
WV    2
DE    1
GA    1
MO    1
PA    1
TX    1
Name: count, dtype: int64
state
MO    3
PA    3
MS    2
OH    2
GA    1
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
state
PA    2
AL    1
OH    1
TX    1
Name: count, dtype: int64
state
GA    1
MO    1
Name: count, dtype: int64
state
IL    2
MS    2
AL    1
NJ    1
Name: count, dtype: int64
state
PA    4
AL    1
FL    1
KY    1
MO    1
NH    1
Name: count, dtype: int64
state
AL    9
GA    8
KY    3
FL    2
NY    2
WI    2
KS    1
OH    1
PA    1
Name: count, dtype: int64
state
FL    8
ND    7
KY    6
PA    5
TX    5
WI    4
AL    3
GA    2
OH    2
OK    2
AR    1
CT    1
MO    1
MS    1
RI    1
VA    1
Name: count, dtype: int64


In [17]:
#Let's try to look for a better outlier.
#What city had the most violations in 2022? And what companies were most responsible?


df["city"][(df["Year"] == 2022)].value_counts().head()

city
HOUSTON         18
PHILADELPHIA    14
LAS VEGAS       12
ANCHORAGE       11
COLUMBUS        11
Name: count, dtype: int64

In [18]:
#It was Houston. Which employer had the most of these penalties?
df["employers_fixed"][
    (df["Year"] == 2022) &
   (df["city"] == "HOUSTON")].value_counts()

employers_fixed
Core Plastech International, Inc.                     1
Gulf Coast Flooring & Services, LLC                   1
Dollar Tree/Family Dollar (similar names merged)      1
Shielder Ventures LLC                                 1
ValvTechnologies LLC                                  1
The Kroger Company                                    1
Kaiser Aluminum & Fabricated Products, LLC            1
American Alloy Steel, Inc.                            1
Alpha Insulation & Waterproofing, Inc.                1
Quality Christmas Tree, Ltd.                          1
Star Pipe Products, LLC                               1
Williams Brothers Construction                        1
Target (similar names merged)                         1
Restaurant Depot Distribution Center #712             1
Melton Electric, Inc.                                 1
Glazier Foods - A Company of Gordon Foods Services    1
TDIndustries, Inc.                                    1
NCI Group, Inc.                 

In [19]:
#Damn, no company had more than one penalty. Let's look at cities per year and states per year. Cities:
print(df["city"][(df["Year"] == 2015)].value_counts().head())
print(df["city"][(df["Year"] == 2016)].value_counts().head())
print(df["city"][(df["Year"] == 2017)].value_counts().head())
print(df["city"][(df["Year"] == 2018)].value_counts().head())
print(df["city"][(df["Year"] == 2019)].value_counts().head())
print(df["city"][(df["Year"] == 2020)].value_counts().head())
print(df["city"][(df["Year"] == 2021)].value_counts().head())
print(df["city"][(df["Year"] == 2022)].value_counts().head())
print(df["city"][(df["Year"] == 2023)].value_counts().head())

city
NEWARK          12
JACKSONVILLE    11
COLUMBUS         8
SAN DIEGO        7
PHILADELPHIA     7
Name: count, dtype: int64
city
HOUSTON         19
LOS ANGELES     14
JACKSONVILLE    13
NEWARK          11
SAN DIEGO       10
Name: count, dtype: int64
city
CHICAGO         25
SAN DIEGO       17
HOUSTON         14
PHILADELPHIA    13
OAKLAND         12
Name: count, dtype: int64
city
SAN DIEGO       21
HOUSTON         14
PHILADELPHIA    12
SEATTLE         11
HONOLULU        10
Name: count, dtype: int64
city
HOUSTON         17
PHILADELPHIA    14
SAN DIEGO       12
ANCHORAGE       12
CINCINNATI      11
Name: count, dtype: int64
city
HOUSTON        14
LAS VEGAS      12
CINCINNATI     11
CHICAGO         9
LOS ANGELES     8
Name: count, dtype: int64
city
HOUSTON        16
COLUMBUS       14
LOS ANGELES    13
MAHWAH         12
PATERSON       10
Name: count, dtype: int64
city
HOUSTON         18
PHILADELPHIA    14
LAS VEGAS       12
ANCHORAGE       11
COLUMBUS        11
Name: count, dtype: int64
ci

In [20]:
#Here are the states with the most incidents per year. Are there any cities that are seeing a rapid increase that
#outpaces other states' rates of increase? If so, perhaps I can look up the state to see if a labor regulation
#changed.
print(df["state"][(df["Year"] == 2015)].value_counts().head(10))
print(df["state"][(df["Year"] == 2016)].value_counts().head(10))
print(df["state"][(df["Year"] == 2017)].value_counts().head(10))
print(df["state"][(df["Year"] == 2018)].value_counts().head(10))
print(df["state"][(df["Year"] == 2019)].value_counts().head(10))
print(df["state"][(df["Year"] == 2020)].value_counts().head(10))
print(df["state"][(df["Year"] == 2021)].value_counts().head(10))
print(df["state"][(df["Year"] == 2022)].value_counts().head(10))
print(df["state"][(df["Year"] == 2023)].value_counts().head(10))

state
CA    145
NJ    110
OH     87
TX     70
IL     64
FL     51
NY     48
GA     42
WI     31
PA     30
Name: count, dtype: int64
state
CA    224
NJ    100
TX     98
OH     82
FL     58
NY     55
PA     53
WA     53
IL     52
GA     50
Name: count, dtype: int64
state
CA    272
IL    119
TX    117
NJ    104
OH     95
NY     75
GA     67
FL     62
MA     48
WA     47
Name: count, dtype: int64
state
CA    268
OH    151
TX    129
NJ    105
IL     82
GA     77
FL     66
PA     55
MA     53
NY     50
Name: count, dtype: int64
state
CA    190
OH    172
TX    125
NJ    114
GA     86
FL     76
IL     76
NY     61
MA     51
MO     49
Name: count, dtype: int64
state
CA    152
OH    133
IL     84
NJ     78
TX     76
FL     56
NY     48
GA     38
MA     37
WA     33
Name: count, dtype: int64
state
CA    193
OH    146
NJ    109
TX     84
WA     62
IL     61
GA     53
NY     48
KS     39
MA     39
Name: count, dtype: int64
state
OH    162
NJ    141
TX    128
CA    104
IL     76
PA     65
WI     55


In [21]:
#Here's an interesting thing: Why did California's drop so greatly last year?
#Let's see if we can find out via the data:

In [22]:
print(df["employers_fixed"][(df["Year"] == 2022) &
   (df["state"] == "CA")].value_counts())

#Nope — no one company was responsible, it seems (unless there's a parent company I'm not
#accounting for.)

employers_fixed
Home Depot U.S.A., Inc.                       2
Spreckels Sugar Company, Inc.                 1
Summit Livestock Facilities, LLC              1
Superior Tree Service Inc.                    1
Total Safety U.S., Inc.                       1
                                             ..
Fairway Staffing Services                     1
The GEO Group, Inc. dba Golden State Annex    1
Darnell Caster LLC                            1
Yuba River Moulding & Mill Work, Inc.         1
PHARAOH STAFFING LLC                          1
Name: count, Length: 103, dtype: int64


In [23]:
#You know what? Let's take the state data and plot it roughly to look for outliers.

In [24]:
#Here's the code to make the DF once we have the data ready.
pd.DataFrame({
    "Year": [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    "CA" : [1, 2, 3, 4, 5, 6, 7, 8, 9],
})

,Year,CA
0,2015,1
1,2016,2
2,2017,3
3,2018,4
4,2019,5
5,2020,6
6,2021,7
7,2022,8
8,2023,9


In [25]:
df_2015 = pd.DataFrame(
    df["state"][(df["Year"] == 2015)].value_counts()
)

In [26]:
df_2016 = pd.DataFrame(
    df["state"][(df["Year"] == 2016)].value_counts()
)

In [27]:
df_2015.merge(df_2016,
             left_on="state",
             right_on="state",
             how="left").head()

,count_x,count_y
state,,
CA,145,224
NJ,110,100
OH,87,82
TX,70,98
IL,64,52


In [28]:
#Let's check that that worked.

In [29]:
pd.DataFrame(
    df["state"][(df["Year"] == 2015)].value_counts()
).head()

,count
state,
CA,145
NJ,110
OH,87
TX,70
IL,64


In [30]:
pd.DataFrame(
    df["state"][(df["Year"] == 2016)].value_counts()
).head()

,count
state,
CA,224
NJ,100
TX,98
OH,82
FL,58


In [31]:
#OK, let's merge 'em all:

merge1 = df_2015.merge(df_2016,
             left_on="state",
             right_on="state",
             how="left")

In [32]:
#Let's also check that it'll eventually be in the right format to merge with the initial "Year" df:
merge1.transpose()

state,CA,NJ,OH,TX,IL,FL,NY,GA,WI,PA,...,AR,VT,AK,TN,NM,MT,SD,MD,DC,AZ
count_x,145,110,87,70,64,51,48,42,31,30,...,3,2,2,2,2,2,1,1,1,1
count_y,224,100,82,98,52,58,55,50,40,53,...,9,1,6,3,4,4,2,6,3,1


In [33]:
#Oh, first we have to make the df for each year:
df_2017 = pd.DataFrame(
    df["state"][(df["Year"] == 2017)].value_counts()
)
df_2018 = pd.DataFrame(
    df["state"][(df["Year"] == 2018)].value_counts()
)
df_2019 = pd.DataFrame(
    df["state"][(df["Year"] == 2019)].value_counts()
)
df_2020 = pd.DataFrame(
    df["state"][(df["Year"] == 2020)].value_counts()
)
df_2021 = pd.DataFrame(
    df["state"][(df["Year"] == 2021)].value_counts()
)
df_2022 = pd.DataFrame(
    df["state"][(df["Year"] == 2022)].value_counts()
)
df_2023 = pd.DataFrame(
    df["state"][(df["Year"] == 2023)].value_counts()
)

In [34]:
#OK, time to continue merging them:

merge2 = merge1.merge(df_2017,
             left_on="state",
             right_on="state",
             how="left")

In [35]:
merge3 = merge2.merge(df_2018,
             left_on="state",
             right_on="state",
             how="left")

MergeError: Passing 'suffixes' which cause duplicate columns {'count_x'} is not allowed.

In [36]:
#Shit. I think it has to do with the number of states varying. Let's check 'em.

In [37]:
print(len(df_2015))
print(len(df_2016))
print(len(df_2017))
print(len(df_2018))
print(len(df_2019))
print(len(df_2020))
print(len(df_2021))
print(len(df_2022))
print(len(df_2023))

50
52
54
48
52
51
54
53
52


In [38]:
#fuck

In [39]:
print(df_2015.reset_index()["state"].value_counts().head())
print(df_2016.reset_index()["state"].value_counts().head())
print(df_2017.reset_index()["state"].value_counts().head())
print(df_2018.reset_index()["state"].value_counts().head())
print(df_2019.reset_index()["state"].value_counts().head())
print(df_2020.reset_index()["state"].value_counts().head())
print(df_2021.reset_index()["state"].value_counts().head())
print(df_2022.reset_index()["state"].value_counts().head())
print(df_2023.reset_index()["state"].value_counts().head())

state
CA    1
RI    1
WY    1
IA    1
IN    1
Name: count, dtype: int64
state
CA    1
NJ    1
NV    1
MD    1
NC    1
Name: count, dtype: int64
state
CA    1
AZ    1
NH    1
SD    1
MS    1
Name: count, dtype: int64
state
CA    1
OH    1
KY    1
OR    1
WV    1
Name: count, dtype: int64
state
CA    1
OH    1
ID    1
MN    1
NE    1
Name: count, dtype: int64
state
CA    1
HI    1
LA    1
VT    1
MI    1
Name: count, dtype: int64
state
CA    1
ME    1
TN    1
NH    1
NM    1
Name: count, dtype: int64
state
OH    1
KY    1
NC    1
ME    1
NH    1
Name: count, dtype: int64
state
OH    1
TX    1
SD    1
IA    1
KS    1
Name: count, dtype: int64


In [40]:
#OK, the variation in the number of states is *not* because of something getting repeated.

In [41]:
#OH, I think I know why I'm getting the merge error. This happened two projects ago.
#I think I need to rename the columns.

In [42]:
merge2.head()

,count_x,count_y,count
state,,,
CA,145,224,272
NJ,110,100,104
OH,87,82,95
TX,70,98,117
IL,64,52,119


In [43]:
merge2 = merge2.rename(columns={'count_x' : 2015, 'count_y' : 2016, 'count' : 2017})

In [44]:
merge3 = merge2.merge(df_2018,
             left_on="state",
             right_on="state",
             how="left")

In [45]:
merge4 = merge3.merge(df_2019,
             left_on="state",
             right_on="state",
             how="left")

In [46]:
merge5 = merge4.merge(df_2020,
             left_on="state",
             right_on="state",
             how="left")

In [47]:
merge5 = merge5.rename(columns={'count_x' : 2018, 'count_y' : 2019, 'count' : 2020})

In [48]:
merge6 = merge5.merge(df_2021,
             left_on="state",
             right_on="state",
             how="left")

In [49]:
merge7 = merge6.merge(df_2022,
             left_on="state",
             right_on="state",
             how="left")

In [50]:
merge8 = merge7.merge(df_2023,
             left_on="state",
             right_on="state",
             how="left")

In [51]:
merge8 = merge8.rename(columns={'count_x' : 2021, 'count_y' : 2022, 'count' : 2023})

In [52]:
df_states = merge8.transpose().reset_index().rename(columns={"index" : "Year"})

In [53]:
df_states
#Let's fix the index

state,Year,CA,NJ,OH,TX,IL,FL,NY,GA,WI,...,AR,VT,AK,TN,NM,MT,SD,MD,DC,AZ
0,2015,145.0,110.0,87.0,70.0,64.0,51.0,48.0,42.0,31.0,...,3.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
1,2016,224.0,100.0,82.0,98.0,52.0,58.0,55.0,50.0,40.0,...,9.0,1.0,6.0,3.0,4.0,4.0,2.0,6.0,3.0,1.0
2,2017,272.0,104.0,95.0,117.0,119.0,62.0,75.0,67.0,40.0,...,15.0,3.0,8.0,2.0,2.0,3.0,7.0,3.0,5.0,3.0
3,2018,268.0,105.0,151.0,129.0,82.0,66.0,50.0,77.0,45.0,...,13.0,4.0,NaN,13.0,NaN,NaN,3.0,6.0,1.0,2.0
4,2019,190.0,114.0,172.0,125.0,76.0,76.0,61.0,86.0,40.0,...,10.0,4.0,24.0,16.0,1.0,1.0,2.0,9.0,2.0,7.0
5,2020,152.0,78.0,133.0,76.0,84.0,56.0,48.0,38.0,30.0,...,20.0,7.0,16.0,6.0,3.0,NaN,4.0,5.0,NaN,3.0
6,2021,193.0,109.0,146.0,84.0,61.0,30.0,48.0,53.0,32.0,...,16.0,4.0,11.0,10.0,8.0,4.0,1.0,19.0,1.0,1.0
7,2022,104.0,141.0,162.0,128.0,76.0,37.0,52.0,45.0,55.0,...,19.0,6.0,19.0,16.0,6.0,3.0,6.0,22.0,3.0,1.0
8,2023,74.0,74.0,127.0,103.0,81.0,48.0,39.0,22.0,51.0,...,24.0,2.0,12.0,8.0,3.0,1.0,10.0,4.0,NaN,2.0


In [54]:
#Tried multiple things and deleted them. It's taking too long. Moving on.

In [55]:
df_states.to_csv('lede_project-4_csv-for-line-chart-test.csv')

In [56]:
#Checked it in Flourish. It turned out too clustered. Let's get straight to the map for now.

In [57]:
df_states_not_transposed = merge8.reset_index().rename(columns={"index" : "Year"})

In [58]:
df_states_not_transposed.head()

,state,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,CA,145,224,272,268.0,190,152.0,193,104,74.0
1,NJ,110,100,104,105.0,114,78.0,109,141,74.0
2,OH,87,82,95,151.0,172,133.0,146,162,127.0
3,TX,70,98,117,129.0,125,76.0,84,128,103.0
4,IL,64,52,119,82.0,76,84.0,61,76,81.0


In [59]:
#To get to the mapping quicker, I'm going to fix isolate it year by year in Google Sheets

In [60]:
df_states_not_transposed.to_csv('lede_project-4_for-map.csv', index=False)

In [61]:
#OK, I made two maps.
#I think I want a line chart of the number of high-fine penalties per year.

In [62]:
print(len(df[df["Year"] == 2015]))
print(len(df[df["Year"] == 2016]))
print(len(df[df["Year"] == 2017]))
print(len(df[df["Year"] == 2018]))
print(len(df[df["Year"] == 2019]))
print(len(df[df["Year"] == 2020]))
print(len(df[df["Year"] == 2021]))
print(len(df[df["Year"] == 2022]))

995
1218
1467
1523
1576
1166
1305
1524


In [63]:
df_ez_linechart = pd.DataFrame({
    2015: [995, 320.738994],
    2016: [1218, 323.071755],
    2017: [1467, 325.122128],
    2018: [1523, 326.838199],
    2019: [1576, 328.329953],
    2020: [1166, 331.511512],
    2021: [1305, 332.031554],
    2022: [1524, 333.287557]
}).transpose().reset_index().rename(columns={"index" : "Year", 0 : "Penalties", 1 : "Population (millions)"})

In [64]:
df_ez_linechart

,Year,Penalties,Population (millions)
0,2015,995.0,320.738994
1,2016,1218.0,323.071755
2,2017,1467.0,325.122128
3,2018,1523.0,326.838199
4,2019,1576.0,328.329953
5,2020,1166.0,331.511512
6,2021,1305.0,332.031554
7,2022,1524.0,333.287557


In [65]:
df_ez_linechart["Penalties per million residents"] = (df_ez_linechart["Penalties"]/df_ez_linechart["Population (millions)"]).round(1)

In [70]:
df_ez_linechart

,Year,Penalties,Population (millions),Penalties per million residents
0,2015,995.0,320.738994,3.1
1,2016,1218.0,323.071755,3.8
2,2017,1467.0,325.122128,4.5
3,2018,1523.0,326.838199,4.7
4,2019,1576.0,328.329953,4.8
5,2020,1166.0,331.511512,3.5
6,2021,1305.0,332.031554,3.9
7,2022,1524.0,333.287557,4.6


In [71]:
df_ez_linechart_accountingforpop = df_ez_linechart.drop(columns=["Penalties", "Population (millions)"])

In [72]:
df_ez_linechart_accountingforpop

,Year,Penalties per million residents
0,2015,3.1
1,2016,3.8
2,2017,4.5
3,2018,4.7
4,2019,4.8
5,2020,3.5
6,2021,3.9
7,2022,4.6


In [75]:
df_ez_linechart_accountingforpop.to_csv("df_ez_linechart_accountingforpop.csv", index=False)